# Process plate counts to get ratios of variants in a re-pooled variant library

Using the fraction of each variant in the initial equal-volume pool, I re-pooled variants so that they might be more balanced. As with the initial pool, I then used the re-pool to infect MDCK-SIAT1 cells. These infections were done by serially diluting a starting 50 uL volume of pool. Barcodes were then isolated and sequenced so that I could confirm equal representation of each variant in the pool, as well as the appropriate library pool dilution (i.e., MOI) to use in neutralization assays. 

The plots generated by this notebook are interactive, so you can mouseover points for details, use the mouse-scroll to zoom and pan, and use interactive dropdowns at the bottom of the plots.

## Setup
Import Python modules:

In [23]:
import pickle
import sys

import altair as alt

import matplotlib.pyplot as plt

import numpy

import pandas as pd
from os.path import join
import os
import ruamel.yaml as yaml

_ = alt.data_transformers.disable_max_rows()

## Add input data locations
Some of these files are defined as data, and some of these files are generated by running the specified library pooling data as `miscellaneous_plates` through the `seqneut-pipeline`. For details on how these files are generated, see the `README.md' in [https://github.com/jbloomlab/seqneut-pipeline](https://github.com/jbloomlab/seqneut-pipeline)

In [24]:
# Define file path prefix
filepath_prefix = '/fh/fast/bloom_j/computational_notebooks/ckikawa/2024/flu_seqneut_H3N2_2023-2024'

# Viral library contents and barcode IDs
viral_library_csv = filepath_prefix + '/data/viral_libraries/2023_H3N2_Kikawa.csv'
# Neutralization standard set of barcode IDs
neut_standard_set_csv = filepath_prefix + '/data/neut_standard_sets/loes2023_neut_standards.csv'
# All samples included in library poolign sequencing run
# Contains information on library, dilution factor, R1 location
samplesfile = filepath_prefix + '/data/miscellaneous_plates/2024-01-22_H3N2_sampleData_rePool_MOItest.csv'

# Counts and fates files output by running library pooling samples as miscellaneous plates
platedir = filepath_prefix + '/results/miscellaneous_plates/240124_repool_H3N2/'

# Identify all counts and fates CSVs
count_csvs = []
fate_csvs = []
file_list = os.listdir(platedir)
for f in file_list:
    location = platedir + f
    if "_counts" in f:
        count_csvs.append(location)
    elif "_fates" in f:
        fate_csvs.append(location)

In [25]:
# Define a samples dataframe using the samples file
samples_df = pd.read_csv(samplesfile)
samples_df.drop(columns=['fastq'], inplace=True)
samples_df['sample'] = samples_df.apply(
    lambda x: '-'.join(x.astype(str)), axis=1
)

samples = samples_df["sample"].unique().tolist()
print(f"There are {len(samples)} barcode runs.")

samples_df

There are 12 barcode runs.


,well,serum,dilution_factor,replicate,sample
0,A1,H3N2library_rePool,1,1,A1-H3N2library_rePool-1-1
1,A2,H3N2library_rePool,2,1,A2-H3N2library_rePool-2-1
2,A3,H3N2library_rePool,4,1,A3-H3N2library_rePool-4-1
3,A4,H3N2library_rePool,8,1,A4-H3N2library_rePool-8-1
4,A5,H3N2library_rePool,16,1,A5-H3N2library_rePool-16-1
5,A6,H3N2library_rePool,32,1,A6-H3N2library_rePool-32-1
6,A7,H3N2library_rePool,64,1,A7-H3N2library_rePool-64-1
7,A8,H3N2library_rePool,128,1,A8-H3N2library_rePool-128-1
8,A9,H3N2library_rePool,256,1,A9-H3N2library_rePool-256-1
9,A10,H3N2library_rePool,512,1,A10-H3N2library_rePool-512-1


## Statistics on barcode-parsing for each sample
Make interactive chart of the "fates" of the sequencing reads parsed for each sample on the plate.

If most sequencing reads are not "valid barcodes", this could potentially indicate some problem in the sequencing or barcode set you are parsing.

Potential fates are:
 - *valid barcode*: barcode that matches a known virus or neutralization standard, we hope most reads are this.
 - *invalid barcode*: a barcode with proper flanking sequences, but does not match a known virus or neutralization standard. If you  have a lot of reads of this type, it is probably a good idea to look at the invalid barcode CSVs (in the `./results/barcode_invalid/` subdirectory created by the pipeline) to see what these invalid barcodes are.
 - *unparseable barcode*: could not parse a barcode from this read as there was not a sequence of the correct length with the appropriate flanking sequence.
 - *low quality barcode*: low-quality or `N` nucleotides in barcode, could indicate problem with sequencing.
 - *failed chastity filter*: reads that failed the Illumina chastity filter, if these are reported in the FASTQ (they may not be).

Also, if the number of reads per sample is very uneven, that could indicate that you did not do a good job of balancing the different samples in the Illumina sequencing.

In [26]:
fates = (
    pd.concat([pd.read_csv(f).assign(well=f.strip(platedir).strip('_fates.csv')) for f, s in zip(fate_csvs, samples)])
    .merge(samples_df, validate="many_to_one", on="well")
    .assign(
        fate_counts=lambda x: x.groupby("fate")["count"].transform("sum"),
        sample_well=lambda x: x["sample"] + " (" + x["well"] + ")",
    )
    .query("fate_counts > 0")[  # only keep fates with at least one count
        ["fate", "count", "well", "sample_well", "dilution_factor"]
    ]
)

assert len(fates) == len(fates.drop_duplicates())


sample_wells = list(
    fates.sort_values(["dilution_factor"])["sample_well"]
)



fates_chart = (
    alt.Chart(fates)
    .encode(
        alt.X("count", scale=alt.Scale(nice=False, padding=3)),
        alt.Y(
            "sample_well",
            title=None,
            sort=sample_wells,
        ),
        alt.Color("fate", sort=sorted(fates["fate"].unique(), reverse=True)),
        alt.Order("fate", sort="descending"),
        tooltip=fates.columns.tolist(),
    )
    .mark_bar(height={"band": 0.85})
    .properties(
        height=alt.Step(10),
        width=200,
        title=f"Barcode parsing for initial titering plate",
    )
    .configure_axis(grid=False)
)

fates_chart

alt.Chart(...)

Potentially, no cDNA was synthesized in well A1? I'll move forward with analysis, but should probably repeat.

## Read barcode counts
Read the counts per barcode:

In [27]:
# get barcode counts
counts = (
    pd.concat([pd.read_csv(c).assign(well=c.strip(platedir).strip('_counts.csv')) for c, s in zip(count_csvs, samples)])
    .merge(samples_df, validate="many_to_one", on="well")
    .drop(columns=["replicate"])
    .assign(sample_well=lambda x: x["sample"] + " (" + x["well"] + ")")
)

# classify barcodes as viral or neut standard
barcode_class = pd.concat(
    [
        pd.read_csv(viral_library_csv)[["barcode", "strain"]].assign(
            neut_standard=False,
        ),
        pd.read_csv(neut_standard_set_csv)[["barcode"]].assign(
            neut_standard=True,
            strain=pd.NA,
        ),
    ],
    ignore_index=True,
)

# merge counts and classification of barcodes
assert set(counts["barcode"]) == set(barcode_class["barcode"])
counts = counts.merge(barcode_class, on="barcode", validate="many_to_one")
assert set(sample_wells) == set(counts["sample_well"])

## Average counts per barcode in each well

Plot average counts per barcode.
If a sample has inadequate barcode counts, it may not have good enough statistics for accurate analysis, and a QC-threshold is applied:

In [28]:
avg_barcode_counts = (
    counts.groupby(
        ["well", "sample_well"],
        dropna=False,
        as_index=False,
    )
    .aggregate(avg_count=pd.NamedAgg("count", "mean"))
    .assign(
        fails_qc=lambda x: (
            x["avg_count"] < 500
        ),
    )
)

avg_barcode_counts_chart = (
    alt.Chart(avg_barcode_counts)
    .encode(
        alt.X(
            "avg_count",
            title="average barcode counts per well",
            scale=alt.Scale(nice=False, padding=3),
        ),
        alt.Y("sample_well", sort=sample_wells),
        alt.Color(
            "fails_qc",
            title=f"fails {'min barcode count threshold'=}",
            legend=alt.Legend(titleLimit=500),
        ),
        tooltip=[
            alt.Tooltip(c, format=".3g") if avg_barcode_counts[c].dtype == float else c
            for c in avg_barcode_counts.columns
        ],
    )
    .mark_bar(height={"band": 0.85})
    .properties(
        height=alt.Step(10),
        width=250,
        title=f"Average barcode counts per well for titering plate",
    )
    .configure_axis(grid=False)
)

display(avg_barcode_counts_chart)

# drop wells failing QC
avg_barcode_counts_per_well_drops = list(avg_barcode_counts.query("fails_qc")["well"])

alt.Chart(...)

As I saw above, there are no barcodes sequenced in well A1. 

## Fraction of counts from neutralization standard
Determine the fraction of counts from the neutralization standard in each sample, and make sure this fraction passess the QC threshold.

In [29]:
neut_standard_fracs = (
    counts.assign(
        neut_standard_count=lambda x: x["count"] * x["neut_standard"].astype(int)
    )
    .groupby(
        ["well", "sample_well", 'dilution_factor'],
        dropna=False,
        as_index=False,
    )
    .aggregate(
        total_count=pd.NamedAgg("count", "sum"),
        neut_standard_count=pd.NamedAgg("neut_standard_count", "sum"),
    )
    .assign(
        neut_standard_frac=lambda x: x["neut_standard_count"] / x["total_count"],
        fails_qc=lambda x: (
            x["neut_standard_frac"] < 0.001
        ),
    )
)

neut_standard_fracs_chart = (
    alt.Chart(neut_standard_fracs)
    .encode(
        alt.X(
            "neut_standard_frac",
            title="frac counts from neutralization standard per well",
            scale=alt.Scale(nice=False, padding=3),
        ),
        alt.Y("sample_well", sort=sample_wells),
        alt.Color(
            "fails_qc",
            title=f"fails {'min_neut_standard_frac_per_well'=}",
            legend=alt.Legend(titleLimit=500),
        ),
        tooltip=[
            alt.Tooltip(c, format=".3g") if neut_standard_fracs[c].dtype == float else c
            for c in neut_standard_fracs.columns
        ],
    )
    .mark_bar(height={"band": 0.85})
    .properties(
        height=alt.Step(10),
        width=250,
        title=f"Neutralization-standard fracs per well for titering plate, initial pool",
    )
    .configure_axis(grid=False)
    .configure_legend(titleLimit=1000)
)

display(neut_standard_fracs_chart)

# drop wells failing QC
min_neut_standard_frac_per_well_drops = list(
    neut_standard_fracs.query("fails_qc")["well"]
)

alt.Chart(...)

In [30]:
# Scatterplot of the same data as above, plotted by dilution factor
alt.Chart(neut_standard_fracs).mark_circle(size=60).encode(
    alt.X('dilution_factor:Q', 
          scale=alt.Scale(type='log'),
          title='library pool reciprocal dilution factor'),
    alt.Y('neut_standard_frac:Q', 
          scale=alt.Scale(type='log'),
          title='fraction of reads = neutralization standard'),
    color='fails_qc',
    tooltip=['well', 'dilution_factor', 'neut_standard_frac', 'total_count']
).interactive()

alt.Chart(...)

It seems as though the first two wells of my dilution series (A1 and B1) are acting sort of weird -- A1 I know had very few barcodes sequenced, but I'm less sure of what could be happening in B1. 

Additionally, between A9 and A10 there is basically no change in fraction neutralization standard. I think there were some technical errors in set-up, and as I said above, I should repeat this.

## Assessing balancing of strains contained in the library
Viruses were rescued and blind passaged individually. Based on data from my initial equal-volume pool, I re-pooled viruses to try to achieve more equal representation. I will now assess that re-pooling. 

Each of the 3 viral barcodes associated with each strain were pooled prior to rescue, so they cannot be balanced. 

In [31]:
# Get summed barcode counts for all strains across all wells
straincounts_allbarcodes = (counts.groupby(['sample','sample_well','strain','dilution_factor','serum','well'])
                          .sum()
                          .reset_index()
                          .drop(columns = ['sample_well', 'neut_standard', 'barcode'])
                         )

# Get sum of all virus/barcode counts per well
sumperwell = (straincounts_allbarcodes.groupby(['sample','dilution_factor','serum','well'])
              .sum()
              .drop(columns=['strain'])
              .reset_index()
              .rename(columns={'count':'counts_perwell'})
             )

# Merge dataframes and calculate fraction of each well devoted to each strain
merged_df = straincounts_allbarcodes.merge(sumperwell, on=['sample','dilution_factor','serum','well'])
merged_df['fraction_strain'] = merged_df['count'] /merged_df['counts_perwell'] 
merged_df

,sample,strain,dilution_factor,serum,well,count,counts_perwell,fraction_strain
0,A1-H3N2library_rePool-1-1,A/Abu_Dhabi/6753/2023,1,H3N2library_rePool,A1,0,8926,0.000000
1,A1-H3N2library_rePool-1-1,A/Bangkok/P3599/2023,1,H3N2library_rePool,A1,0,8926,0.000000
2,A1-H3N2library_rePool-1-1,A/Bangkok/P3755/2023,1,H3N2library_rePool,A1,236,8926,0.026440
3,A1-H3N2library_rePool-1-1,A/Bhutan/0006/2023,1,H3N2library_rePool,A1,2317,8926,0.259579
4,A1-H3N2library_rePool-1-1,A/Bhutan/0845/2023,1,H3N2library_rePool,A1,0,8926,0.000000
...,...,...,...,...,...,...,...,...
907,A9-H3N2library_rePool-256-1,A/Texas/50/2012_X-223A_(13/252),256,H3N2library_rePool,A9,5918,663665,0.008917
908,A9-H3N2library_rePool-256-1,A/Townsville/68/2023,256,H3N2library_rePool,A9,9419,663665,0.014192
909,A9-H3N2library_rePool-256-1,A/Victoria/1033/2023,256,H3N2library_rePool,A9,6885,663665,0.010374
910,A9-H3N2library_rePool-256-1,A/Wisconsin/27/2023,256,H3N2library_rePool,A9,7455,663665,0.011233


We now have this fraction of reads devoted to all strains calculated for all wells. However, ideally we should just focus on those wells containing dilutions that we would use for actual neutralization assays. We should choose a set of replicate wells where the fraction of neutralization standard reads begins to increase linearly with the increasing reciprocal dilution factor. See plots above for choosing these wells. 

In [32]:
# Using A9 and B9, corresponding to reciprocal dilution factor = 256
single_well = merged_df.loc[merged_df['sample'].str.contains('A9-|B9-')]

In [33]:
# Calculate mean fraction strain across both wells
mean_df = single_well.groupby(['strain'])['fraction_strain'].mean().to_frame().rename(columns = {'fraction_strain': 'mean_fraction_strains'}).reset_index()
mean_single_well = single_well.merge(mean_df, on = 'strain', how = 'left')

# calcualte ratios to add for equal pool
num_strains = 76
mean_single_well['ratio_to_add'] = (1/num_strains)/mean_single_well['fraction_strain']
mean_single_well['mean_ratio_to_add'] = (1/num_strains)/mean_single_well['mean_fraction_strains']

mean_single_well['est_tcid50'] = (mean_single_well['mean_fraction_strains']*25000)*76

mean_single_well.head(5)

,sample,strain,dilution_factor,serum,well,count,counts_perwell,fraction_strain,mean_fraction_strains,ratio_to_add,mean_ratio_to_add,est_tcid50
0,A9-H3N2library_rePool-256-1,A/Abu_Dhabi/6753/2023,256,H3N2library_rePool,A9,5496,663665,0.008281,0.008281,1.588871,1.588871,15734.444336
1,A9-H3N2library_rePool-256-1,A/Bangkok/P3599/2023,256,H3N2library_rePool,A9,7142,663665,0.010761,0.010761,1.222688,1.222688,20446.761544
2,A9-H3N2library_rePool-256-1,A/Bangkok/P3755/2023,256,H3N2library_rePool,A9,15958,663665,0.024045,0.024045,0.547214,0.547214,45686.001220
3,A9-H3N2library_rePool-256-1,A/Bhutan/0006/2023,256,H3N2library_rePool,A9,5203,663665,0.007840,0.007840,1.678346,1.678346,14895.617518
4,A9-H3N2library_rePool-256-1,A/Bhutan/0845/2023,256,H3N2library_rePool,A9,5892,663665,0.008878,0.008878,1.482083,1.482083,16868.148840


## Visualize barcode- and strain-level balancing in the current pool

In [38]:
# Plot the current fraction of each strain in the pool
strains_chart = (
    alt.Chart(mean_single_well)
    .encode(
        alt.X(
            "fraction_strain",
            scale=alt.Scale(nice=False, padding=3),
        ),
        alt.Y("strain"),
        
        tooltip = ['strain', 'fraction_strain', 'est_tcid50'],
    )
).mark_bar(height={"band": 0.85}).properties(
        height=alt.Step(10),
        width=250,
        title="",
    ).properties(
        height = alt.Step(10),
        width = 200,
        title = "Strain representation, initial pool")

# add veritcal line where we would expect equal representation of all barcodes in pool
expected_line = alt.Chart(
    pd.DataFrame({'x': [1/75]})
).mark_rule(strokeDash = [2,2], strokeWidth = 2).encode(x = 'x')

# plot both barcode counts and expected line
strains_chart + expected_line

alt.LayerChart(...)

One nice piece of data from this otherwise sort of bust of an experiment -- pooling looks good! Strains are all well-represented except for A/Kansas/14/2017_X-327, which was dropped due to extremely low titers. Almost all strains are within 2-fold of the dotted line, which indicates the fraction where the pool has true equal representation.

In [35]:
# Each barcode fraction across strains
all_barcode_counts = counts[['strain', 'barcode', 'count', 'well']].dropna()
single_well_all_barcode_counts = all_barcode_counts[all_barcode_counts['well'].isin(['A9','B9'])]

# Get tidy single well means
tidy_single_well = single_well_all_barcode_counts[['strain','barcode','count']].groupby(['strain', 'barcode']).mean().reset_index()
# Get sums for each strain
strain_sums_df = tidy_single_well.groupby('strain').sum().rename(columns = {'count': 'strain_count_sum'}).reset_index()
# Merge and calculate per strain the fraction represented by each barcode
tidy_single_well = tidy_single_well.merge(strain_sums_df[['strain', 'strain_count_sum']], 
                       on = ['strain'],
                       validate="many_to_one",
                      )
tidy_single_well['per_strain_fraction_barcode'] = tidy_single_well['count'] / tidy_single_well['strain_count_sum']
tidy_single_well['barcode_letter'] = (['A', 'B', 'C'] * len(strain_sums_df))

# Plot as colored bar chart
bar_chart = alt.Chart(tidy_single_well).mark_bar(height={"band": 0.85}).encode(
    x = 'per_strain_fraction_barcode',
    y = 'strain',
    color=alt.Color('barcode_letter', legend=None),
    tooltip = ['strain', 'per_strain_fraction_barcode', 'barcode'],
).configure_axis(grid=False).properties(
        height = alt.Step(10),
        width = 200,
        title = "Barcode fraction for each strain, initial pool")

bar_chart

alt.Chart(...)